# Everything is on the cloud now and I feel numb

There's something off about the distance calculation 

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import matplotlib.colors as mcolors

In [2]:
# the location of our credentials json and name of bigquery project
credentials = service_account.Credentials.from_service_account_file('C:\\Users\\elija\\Documents\\24f-coop\\credentials.json')
project = 'net-data-viz-handbook'

# Initialize a GCS client
client = bigquery.Client(credentials=credentials, project=project)

## Getting df_pivot for graphing porpoises
This will happen 1) much later in the process and 2) on Google Cloud, but I just want to have easy access to it for now. 

In [5]:
# df_pivot just because it's nice to have
query_pivot = """
SELECT *
FROM (
    SELECT 
        date,
        country_id,
        run_id,
        SUM(Infectious_13_17 + Infectious_18_23) AS total_infectious
    FROM `net-data-viz-handbook.sri_data.SIR_0_countries_incidence_daily`
    WHERE country_id IN (215)
      AND run_id BETWEEN 1 AND 100
    GROUP BY date, country_id, run_id
)
PIVOT (
    SUM(total_infectious) FOR run_id IN (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
                                         21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
                                         41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 
                                         61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 
                                         81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100)
)
ORDER BY date;
"""

In [6]:
# Run the query
query_job = client.query(query_pivot)

# Fetch the results into a DataFrame
df_pivot = query_job.to_dataframe()

del df_pivot['country_id']
df_pivot.set_index('date', inplace=True)

# this is gonna be hell in SQL
df_pivot.columns = df_pivot.columns.str.replace('_', '', regex=False).astype(int)

# Display the first few rows
df_pivot.head()

C:\Users\elija\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
date,,,,,,,,,,,,,,,,,,,,,
2009-02-17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2009-02-18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2009-02-19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2009-02-20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2009-02-21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## KMeans and a prayer
- Uses the first num_features columns to simplify the KMeans algorithms, which is O(n<sup>2</sup>)
- Procedural once distance matrix is computed (eg have to query main table manually)

In [7]:
# Step 1: Create the MSE matrix table
query_mse_matrix = """
CREATE OR REPLACE TABLE sri_data.mse_matrix AS
WITH infectious_data AS (
    SELECT
        date,
        country_id,
        run_id,
        SUM(Infectious_13_17 + Infectious_18_23) AS total_infectious
    FROM
        net-data-viz-handbook.sri_data.SIR_0_countries_incidence_daily
    WHERE
        country_id IN (215) AND run_id BETWEEN 1 AND 100
    GROUP BY
        date,
        country_id,
        run_id
)
SELECT
    a.run_id AS run_id_a,
    b.run_id AS run_id_b,
    AVG(POW(a.total_infectious - b.total_infectious, 2)) AS mse
FROM
    infectious_data a
JOIN
    infectious_data b
ON
    a.date = b.date
GROUP BY
    run_id_a, run_id_b
"""
client.query(query_mse_matrix).result()  # Execute the query to create the table
print("MSE matrix table created successfully.")

# Step 2: Create the ABC matrix table
query_abc_matrix = """
CREATE OR REPLACE TABLE sri_data.abc_matrix AS
WITH infectious_data AS (
    SELECT
        date,
        country_id,
        run_id,
        SUM(Infectious_13_17 + Infectious_18_23) AS total_infectious
    FROM
        net-data-viz-handbook.sri_data.SIR_0_countries_incidence_daily
    WHERE
        country_id IN (215) AND run_id BETWEEN 1 AND 100
    GROUP BY
        date,
        country_id,
        run_id
)
SELECT
    a.run_id AS run_id_a,
    b.run_id AS run_id_b,
    SUM(ABS(a.total_infectious - b.total_infectious)) AS abc
FROM
    infectious_data a
JOIN
    infectious_data b
ON
    a.date = b.date
GROUP BY
    run_id_a, run_id_b
"""
client.query(query_abc_matrix).result()  # Execute the query to create the table
print("ABC matrix table created successfully.")

MSE matrix table created successfully.
ABC matrix table created successfully.


In [51]:
# Step 3: Create the distance matrix
method = 'mse' # 'mse' or 'abc'

query_distance_matrix = f"""
CREATE OR REPLACE TABLE sri_data.distance_matrix AS
SELECT
    run_id_a AS run_id,
        ARRAY_AGG(STRUCT(run_id_b, {method}) ORDER BY run_id_b ASC) AS distances
FROM
    sri_data.{method}_matrix
GROUP BY
    run_id_a;
"""
client.query(query_distance_matrix).result()  # Execute the query to create the table
print("Distance matrix table created successfully.")

Distance matrix table created successfully.


In [ ]:
# Set parameters for the K-means model
num_clusters = 2  # Example number of clusters
num_features = 5   # Set the number of features to select

In [ ]:
# Step 4: Create the K-means model by selecting the first num_features features based on actual distances
query_create_model = f"""
CREATE OR REPLACE MODEL sri_data.kmeans_model
OPTIONS(model_type='kmeans', num_clusters={num_clusters}) AS
SELECT
    run_id,
    ARRAY(
        SELECT distance.{method} 
        FROM UNNEST(distances) AS distance 
        WHERE distance.run_id_b <= {num_features}  -- Select only the first num_features
    ) AS features
FROM
    sri_data.distance_matrix;
"""
s = time.time()
client.query(query_create_model).result()  # Execute the model creation
print(f"K-means model created successfully in {round(time.time() - s, 3)} seconds.")

# Step 5: Apply K-means clustering and save results in a table
query_kmeans = f"""
CREATE OR REPLACE TABLE sri_data.kmeans_results AS
SELECT
    *
FROM
    ML.PREDICT(MODEL sri_data.kmeans_model,
        (SELECT
            run_id,
            ARRAY(
                SELECT distance.{method} 
                FROM UNNEST(distances) AS distance 
                WHERE distance.run_id_b <= {num_features}  
            ) AS features
         FROM
            sri_data.distance_matrix)
    ) AS predictions
"""
s = time.time()
client.query(query_kmeans).result()  # Execute the model creation
print(f"K-means clustering results saved successfully in {round(time.time() - s, 3)} seconds.")

In [ ]:
# Step 6: Sum distances within each cluster, second variation
# 
percentile = 1 # not implemented currently, will let you select only most central subset of data

query_sum_distances = f"""
WITH a AS (
    SELECT 
        kr.CENTROID_ID,
        kr.run_id, 
        run_id_b, 
        {method}
    FROM 
        `sri_data.kmeans_results` AS kr
    JOIN 
        `sri_data.distance_matrix` AS dm
    ON 
        kr.run_id = dm.run_id
    CROSS JOIN 
        UNNEST(dm.distances) AS dm_dist  -- Unnest the distances array here 
),
b AS (
    SELECT
        run_id AS run_id_b, 
        CENTROID_ID AS CENTROID_ID_B
    FROM
        `sri_data.kmeans_results`
)
SELECT 
    a.run_id,
    a.CENTROID_ID,
    AVG({method}) as total_distance
FROM 
    a
JOIN 
    b
ON 
    a.run_id_b = b.run_id_b
WHERE
    a.CENTROID_ID = b.CENTROID_ID_B
GROUP BY
    a.CENTROID_ID,
    a.run_id;

"""
total_distances = client.query(query_sum_distances).to_dataframe()  # Execute and fetch results
print("Total distances summed within each cluster successfully.")

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Create a figure
fig = go.Figure()

# Define a list of colors for distinct centroids
colors = [
    'rgba(128, 0, 0, {alpha})',    # Red
    'rgba(0, 0, 128, {alpha})',    # Blue
    'rgba(0, 128, 0, {alpha})',    # Green
    'rgba(128, 0, 128, {alpha})',  # Orange
    'rgba(255, 0, 255, {alpha})',  # Magenta
    'rgba(0, 255, 255, {alpha})',  # Cyan
    'rgba(128, 0, 128, {alpha})',  # Purple
    # Add more colors as needed
]

# Loop through each run_id to plot the curves
for index, row in total_distances.iterrows():
    run_id = row['run_id']
    centroid_id = row['CENTROID_ID']
    total_distance = row['total_distance']

    # Get the curve data for the run_id from df_pivot
    curve_data = df_pivot[run_id].rolling(7).median()

    # Calculate alpha value based on total distance
    alpha_value = max(0.1, 1 - (total_distance / total_distances['total_distance'].max()))  # Ensure alpha is between 0 and 1

    # Select a color for the centroid
    color = colors[centroid_id % len(colors) - 1].format(alpha=alpha_value)  # Loop through colors based on centroid_id

    # Add the curve to the figure
    fig.add_trace(go.Scatter(
        x=curve_data.index,  # Assuming the x-axis is the index of df_pivot
        y=curve_data.values,  # y-values from df_pivot
        mode='lines',
        name=f'Run ID {run_id}',
        line=dict(color=color),  # Use the selected color
        showlegend=False
    ))

# Update layout
fig.update_layout(
    title='Curves by Centroid ID with Distance-based Alpha',
    xaxis_title='Date',  # Change this to your desired x-axis label
    yaxis_title='Incidence',  # Change this to your desired y-axis label
    legend_title='Run ID',
)

# Show the figure
fig.show()


In [ ]:
def color_to_rgba(color_name, value, alpha):
    # Get the RGB components of the color
    rgb = mcolors.to_rgb(color_name)
    
    # Calculate the gray value based on the input value
    gray = (1 - value) * 0.8  # This adjusts how gray the color will be

    # Compute the final RGBA values
    r = rgb[0] * value + gray
    g = rgb[1] * value + gray
    b = rgb[2] * value + gray
    
    # Return the RGBA string
#     print(f'rgba({int(r * 255)}, {int(g * 255)}, {int(b * 255)}, {alpha})')
    return f'rgba({int(r * 255)}, {int(g * 255)}, {int(b * 255)}, {alpha})'

## Making a Functional Boxplot
This will be done group-wise (and in SQL, but I wanted to make sure it worked for today)

In [ ]:
# create and lay out graph
fig = go.Figure()
fig.update_layout(
    title={
        'text': f"<b>Functional Boxplot</b><br><span style='font-size: 12px;'>As seen in \
<a href=https://www.tandfonline.com/doi/pdf/10.1198/jcgs.2011.09224?casa_token=ID3IjHflKz4AAAAA:4i-zhPbXhDzg\
8pDuowEPWoNiUFzHFcADAHHsqPonc6ac4dIzuQ40g5VA_n4BlUU7v1JsW7OD7Hf2>Sun & Genton (2011)</a>.  \
Clusters: {num_clusters}, features: {num_features}, method: {method}.</span>",
        'x': 0.5,  
        'y': 0.9,  
    },
    xaxis_title="Date",
    yaxis_title="Incidence",

)
fig.update_xaxes(range=[pd.Timestamp("2009-09-01"), pd.Timestamp("2010-02-17")])

# prepare graphing resources
colors = 'maroon', 'navy', 'green', 'purple'
df_graph = df_pivot.rolling(7).median()

# do a bunch of data processing to get quartiles, will happen on SQL
for group in total_distances['CENTROID_ID'].unique():
    print("Group", group)
    df_group = total_distances[total_distances['CENTROID_ID'] == group]
    print('Total Curves:', len(df_group))
    
    # Calculate quartiles
    lower_quartile = df_group['total_distance'].quantile(.25)
    upper_quartile = df_group['total_distance'].quantile(.75)
    
    # Calculate IQR
    IQR = upper_quartile - lower_quartile
    print('IQR:', round(IQR))
    
    # Filter rows within IQR (inner curves)
    inner_curves = df_group.sort_values('total_distance').iloc[:int(len(df_group)/2), 0].to_list()
    print('Inner Curves:', len(inner_curves))
    print(inner_curves)
    median = df_group.sort_values('total_distance').iloc[0, 0]
    print('Median Curve:', median)
    
    curve_75 = df_graph[inner_curves].T.max()
    curve_25 = df_graph[inner_curves].T.min()
    
    # Define outliers (beyond 1.5 * IQR)
    lower_bound = lower_quartile - 1.5 * IQR
    upper_bound = upper_quartile + 1.5 * IQR
    print('Bounds:', round(lower_bound), round(upper_bound))
    
    # Filter to get outliers (those outside 1.5 IQRs)
    outliers = df_group[(df_group['total_distance'] < lower_bound) | \
                        (df_group['total_distance'] > upper_bound)]['run_id'].to_list()
    print('Outliers:', len(outliers))
    print(outliers, '\n')
    
    # Now calculate the curve_0 and curve_100, excluding outliers
    non_outliers = df_group[~df_group['run_id'].isin(outliers)]['run_id'].to_list()
    
    curve_0 = df_graph[non_outliers].T.min()
    curve_100 = df_graph[non_outliers].T.max()
    
    # actually graph
    # Lower
    fig.add_trace(go.Scatter(
        name=f'Group {group} Lower Bound',
        x=curve_0.index,
        y=curve_0,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        showlegend=False,
        legendgroup=str(group)  # Assign to legend group
    ))
    # Upper
    fig.add_trace(go.Scatter(
        name=f'Group {group} Upper Bound',
        x=curve_100.index,
        y=curve_100,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        fillcolor=color_to_rgba(colors[group-1], 1, .2),
        fill='tonexty',
        showlegend=False,
        legendgroup=str(group)  # Assign to legend group
    ))
        
    # Lower
    fig.add_trace(go.Scatter(
        name=f'Group {group} Lower Quartile',
        x=curve_25.index,
        y=curve_25,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        showlegend=False,
        legendgroup=str(group)  # Assign to legend group
    ))
    # Upper
    fig.add_trace(go.Scatter(
        name=f'Group {group}',
        x=curve_75.index,
        y=curve_75,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        fillcolor=color_to_rgba(colors[group-1], 1, alpha=.2),
        fill='tonexty',
        showlegend=True,
        legendgroup=str(group)  # Assign to legend group
    ))
    
    for outlier in outliers:
        fig.add_trace(go.Scatter(
        name=f'Group {group} Outlier',
        x=df_graph.index,
        y=df_graph[outlier],
        marker=dict(color=color_to_rgba(colors[group-1], .5, alpha=.1)),
        line=dict(width=1, dash='solid'),
        mode='lines',
        showlegend=False,
        legendgroup=str(group)  # Assign to legend group
    ))
    
    fig.add_trace(go.Scatter(
        name=f'Group {group} Median',
        x=df_graph.index,
        y=df_graph[median],
        marker=dict(color=color_to_rgba(colors[group-1], 1, alpha=1)),
        line=dict(width=1),
        mode='lines',
        showlegend=False,
        legendgroup=str(group)  # Assign to legend group
    ))
    
        

        
fig.show()

In [ ]:
plt.plot(df_pivot.rolling(7).median(), alpha=.5, lw=.5, c='gray'); # remove this semicolon at your own risk

In [ ]:
total_distances

## KMeans runtime testing

In [ ]:
import time
import plotly.express as px
import pandas as pd
from tqdm.notebook import tqdm

# Sample variables
ls_num_clusters = [1, 2, 3, 10, 50]
ls_num_features = [1, 3, 5, 10, 50, 100]
i = 1  # Number of iterations

# Dictionary to store runtimes
runtime_data = {}

# Loop over the number of iterations
for num_clusters in ls_num_clusters:
    for num_features in ls_num_features:
        for _ in tqdm(range(i), desc=f'Clusters: {num_clusters} | Features: {num_features}', leave=False):
            start_time = time.time()
            # Simulate the query execution (replace this with your actual query)
            client.query(query_create_model).result()  # Replace with actual query
            end_time = time.time()
            
            runtime = end_time - start_time
            
            # Store runtime in the dictionary
            key = (num_clusters, num_features)
            if key in runtime_data:
                runtime_data[key].append(runtime)
            else:
                runtime_data[key] = [runtime]

# Calculate average runtimes
average_runtimes = {
    'num_clusters': [],
    'num_features': [],
    'average_runtime': []
}

for (num_clusters, num_features), runtimes in runtime_data.items():
    average_runtimes['num_clusters'].append(num_clusters)
    average_runtimes['num_features'].append(num_features)
    average_runtimes['average_runtime'].append(sum(runtimes) / len(runtimes))